In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import pandas as pd
from pprint import pprint
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import torch
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import gensim
from gensim.models.fasttext import FastText
from gensim.models.wrappers import FastText
from gensim.test.utils import datapath
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models import KeyedVectors

# encoder_path = './encoder_hs1000es300p1100.pt'
# decoder_path = './decoder_hs1000es300p1100.pt'
# model_path = './model_hs1000es300p1100.pt'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

In [ ]:
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
train = train.dropna()

In [ ]:
train_pairs = train[['text','selected_text']]
train_pairs = train_pairs.values.tolist()

In [ ]:
vocab_df = pd.concat([train['text'],test['text']],axis=0,ignore_index=True)
vocab_list = vocab_df.values.tolist()

In [ ]:
test_df = test['text']
test_list = test_df.values.tolist()

In [ ]:
SOS_token = 0
EOS_token = 34

sos_tensor = torch.tensor(SOS_token, dtype=torch.long, device=device)
eos_tensor = torch.tensor(EOS_token, dtype=torch.long, device=device)

# class Lang:
#     def __init__(self, name):
#         self.name = name
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {0: "SOS", 1: "EOS"}
#         self.n_words = 2  # Count SOS and EOS
        
#     def addSentence(self, sentence):
#         for word in sentence.split(' '):
#             self.addWord(word)

#     def addWord(self, word):
#         if word not in self.word2index:
#             self.word2index[word] = self.n_words
#             self.word2count[word] = 1
#             self.index2word[self.n_words] = word
#             self.n_words += 1
#         else:
#             self.word2count[word] += 1

In [ ]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def normalizeString(s):
#     s = re.sub(r'https?:\/\/.\S+', "", s)
#     s = re.sub(r'[-*0123456789¿½()"]', ' ', s)
    s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
#     s = " ".join([asd for asd in re.split("([A-Z][a-z]+[^A-Z]*)",s) if asd])
    s = s.strip()
    s = s.split()
    return " ".join(s)

In [ ]:
vocab_list = [normalizeString(l) for l in vocab_list]
test_list = [normalizeString(l) for l in test_list]
train_pairs = [[normalizeString(s) for s in l] for l in train_pairs]

In [ ]:
# def readLangs(lang:str):
#     pairs = train_pairs 
#     vocab_lang = Lang(lang)
#     return vocab_lang, pairs

In [ ]:
# def prepareData(lang:str):
#     vocab_lang, pairs = readLangs(lang)
    
#     print("Read %s sentence pairs" % len(pairs))
#     print("Counting words...")
#     for sentence in vocab_list:
#         vocab_lang.addSentence(sentence)
#     for sentence_pair in pairs:
#         for sentence1 in sentence_pair:
#             vocab_lang.addSentence(sentence1)
        
#     print("Counted words:")
#     print(vocab_lang.name, vocab_lang.n_words)
#     return vocab_lang, pairs


# vocab,  pairs = prepareData('vocab')

# print(random.choice(pairs))

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, dropout_odds=0.3):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(dropout_odds)
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.embedding.weight.requires_grad = True
        self.gru = nn.GRU(embedding_size, hidden_size, bidirectional=True)
        self.fc_hidden = nn.Linear(hidden_size*2, hidden_size)
        
    def forward(self, x):
        embedded = self.dropout(self.embedding(x)).unsqueeze(1)
        output, hidden = self.gru(embedded)
        hidden = self.fc_hidden(torch.cat((hidden[0:1],hidden[1:2]),dim=2))
        
        return output, hidden

In [ ]:
# a,b,c = tensorFromPair(train_pairs[0][0])
# hidden_size = 256
# embedding_size=300
# vocab_size = 35
# dropout_odds = 0.3
# encoder = Encoder(vocab_size, embedding_size, hidden_size).to(device)
# encoder_outputs,hidden = encoder(a[0])
# # dropout = nn.Dropout(dropout_odds).to(device)
# # embedding = nn.Embedding(35, embedding_size).to(device)
# # gru = nn.GRU(embedding_size, hidden_size, bidirectional=True).to(device)
# # fc_hidden = nn.Linear(hidden_size*2, hidden_size).to(device)
# # x = training_pairs[0][0]

# # zxc = embedding(x)
# # embedded = dropout(embedding(x)).unsqueeze(1)
# # output, hidden = gru(embedded)
# # hidden = fc_hidden(torch.cat((hidden[0:1],hidden[1:2]),dim=2))

In [ ]:
# embedding = nn.Embedding(vocab_size, embedding_size).to(device)
# energy = nn.Linear(hidden_size*3, 1).to(device)
# gru = nn.GRU(hidden_size*2+embedding_size, hidden_size).to(device)
# out = nn.Linear(hidden_size, vocab_size).to(device)
# relu = nn.ReLU().to(device)
# softmax_en = nn.Softmax(dim=0).to(device)
# x = sos_tensor
# target = a[1]
# target_len = target.shape[0]
# decoder_outputs = torch.zeros(target_len, 35).to(device)
# eos_loss_tensor = torch.tensor([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,100]).to(device)
# teacher_force_ratio = 0.5

# embedded = embedding(x).unsqueeze(0).unsqueeze(0)
# sequence_length = encoder_outputs.shape[0]
# h_reshaped = hidden[0].repeat(sequence_length, 1,1)
# energy_ = relu(energy(torch.cat((h_reshaped, encoder_outputs), dim=2)))
# attention = softmax_en(energy_)
# context_vector = torch.einsum("snk,snl->knl", attention, encoder_outputs)
# rnn_input = torch.cat((context_vector, embedded),dim=2)
# output, hidden = gru(rnn_input, hidden)
# output = relu(out(hidden)).squeeze(0)
# if output.argmax() not in target[:-1]:
#     output = torch.ger(output.squeeze(0),eos_loss_tensor.float())[1]
# decoder_outputs[1] = output
# best_guess = output.argmax()
# x = target[1] if random.random() < teacher_force_ratio else best_guess
# print(output)
# print(output.argmax())
# print(hidden.shape)
# print(outVector2wordList(output,c))

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, dropout_odds=0.3):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(dropout_odds)
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.embedding.weight.requires_grad = True
        self.energy = nn.Linear(hidden_size*3, 1)
        self.gru = nn.GRU(hidden_size*2+embedding_size, hidden_size)
        self.out = nn.Linear(hidden_size, 35)
        self.relu = nn.ReLU()
        self.softmax_en = nn.Softmax(dim=0)


    def forward(self, x, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(x)).unsqueeze(0).unsqueeze(0)
        sequence_length = encoder_outputs.shape[0]
        h_reshaped = hidden[0].repeat(sequence_length, 1,1)
        energy_ = self.relu(self.energy(torch.cat((h_reshaped, encoder_outputs), dim=2)))
        attention = self.softmax_en(energy_)
        context_vector = torch.einsum("snk,snl->knl", attention, encoder_outputs)
        rnn_input = torch.cat((context_vector, embedded),dim=2)
        output, hidden = self.gru(rnn_input, hidden)
        
        output = self.relu(self.out(hidden)).squeeze(0)
        return output, hidden

In [ ]:
class Seq2seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, source, target, teacher_force_ratio=0.5, output_size = 35):
        sourse_len = source.shape[0]
        target_len = target.shape[0]
#         eos_loss_tensor = torch.tensor([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,100]).to(device)
#         target_vocab_size = vocab.n_words
#         hidden = torch.zeros(hiden_size)
#         encoder_outputs =  torch.zeros(sourse_len,1, hidden_size*2).to(device)
        decoder_outputs = torch.zeros(target_len, output_size).to(device)
#         attention_outputs = torch.zeros(target_len,sourse_len).to(device)
#         for ei in range(sourse_len):
        encoder_states, hidden = self.encoder(source)
#             encoder_outputs[ei] = encoder_states
        x = sos_tensor

        for t in range(target_len):
#             output, hidden, _ = self.decoder(x, hidden, encoder_outputs)
            output, hidden = self.decoder(x, hidden, encoder_states)
            decoder_outputs[t] = output
            best_guess = output.argmax()
            x = target[t] if random.random() < teacher_force_ratio else best_guess
            

        return decoder_outputs
        

In [ ]:
# def indexFromSentence(sentence):
#     return [vocab.word2index[word] for word in sentence.split(' ')]

# def tensorFromIndex(sentance):
#     indexes = indexFromSentence(sentance)
#     indexes.append(EOS_token)
#     return torch.tensor(indexes, dtype=torch.long, device=device)   
    
# def tensorsFromPair(pair):
#     input_tensor = tensorFromIndex(pair[0])
#     target_tensor = tensorFromIndex(pair[1])
#     return (input_tensor, target_tensor)

In [ ]:
def indexFromInput(sentence):
    i=1
    word2idx_voc = {'SOS':0,'EOS':34}
    idx2word_voc = {0:'SOS',34:'EOS'}
    idx_list = []
    for word in sentence.split(' '):
        if word not in word2idx_voc:
            word2idx_voc[word] = i
            idx2word_voc[i] = word
        idx_list.append(i)
        i+=1
    idx_list.append(34)
    return word2idx_voc, idx_list, idx2word_voc

def indexFromPair(pair):
    j=1
    trg_lis = []
    w2ivoc, inp_lis, i2wvoc = indexFromInput(pair[0])
    for word in pair[1].split(' '):
        try:
            trg_lis.append(voc[word])
            j=voc[word]
        except:
            trg_lis.append(j+1)
            j+=1
    trg_lis.append(34)    
    return inp_lis, trg_lis, w2ivoc, i2wvoc
        

def tensorFromPair(pair):
    lisInput,lisTarget,w2ivoc, i2wvoc = indexFromPair(pair)
    inp = torch.tensor(lisInput, dtype=torch.long, device=device)
    tar = torch.tensor(lisTarget, dtype=torch.long, device=device)
    return (inp,tar), w2ivoc, i2wvoc

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


# def timeSince(since, percent):
def timeSince(since):
    now = time.time()
    s = now - since
#     es = s / (percent)
#     rs = es - s
#     return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
    return asMinutes(s)

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
def outVector2wordList(out,vocab):
    list1 = []
    for i in out.argmax(1):
        try:
            list1.append(vocab[i.item()])
        except:
            list1.append('TUNK')
    return ' '.join(list1)

In [ ]:
def targetVector2wordsList(trgt,vocab):
    list2 = []
    for i in range(len(trgt)):
        try:
            list2.append(vocab[trgt[i].item()])
        except:
            list2.append('VUNK')
    return ' '.join(list2)

In [ ]:
hidden_size = 512
embedding_size = 300
vocab_size = 35
epochs = 5
patience_s = 100
weight_list = [1,1,1,1,1,1,0.9,0.9,0.9,0.9,0.9,0.9,0.7,0.7,0.7,0.7,0.7,0.7,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.1]
weight_tensor = torch.tensor(weight_list,dtype=torch.float).to(device)
encoder = Encoder(vocab_size, embedding_size, hidden_size).to(device)
# encoder.load_state_dict(torch.load(encoder_path,map_location=torch.device('cpu')))
decoder = Decoder(vocab_size, embedding_size, hidden_size).to(device)
# decoder.load_state_dict(torch.load(decoder_path,map_location=torch.device('cpu')))
model = Seq2seq(encoder,decoder).to(device)
# model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
optimizer = optim.Adam(model.parameters(), lr=0.00001)
# optimizer = Adam16(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.1, verbose=True)
# class_weights = torch.tensor(weights,dtype=torch.float16).to(device)
criterion = nn.CrossEntropyLoss(weight=weight_tensor).to(device)
# training_pairs = [tensorFromPair(pair) for pair in train_pairs[:2200]]
# model.eval()
# encoder.train()
# decoder.train()

In [ ]:
rubicon = 27000
start = time.time()
loss1=9999
loss2=9999
indx = 0
for epoch in range(1,epochs+1):
    model.train()
    print(f"[Epoch {epoch} / {epochs}]")
    loss_sum = 0
    loss_sum_valid = 0
    jaccard_score_train = 0
    jaccard_score_valid = 0
        
    for t_pair in train_pairs[:rubicon]:
        tp,w2ivocab,i2wvocab = tensorFromPair(t_pair)
        inp_data = tp[0]
        target = tp[1]
        output = model(inp_data, target)
        optimizer.zero_grad()
        loss = criterion(output, target)
        loss_sum +=loss.item()
        loss.backward()
        optimizer.step()
        jaccard_score_train += jaccard(outVector2wordList(output[:-1],i2wvocab),t_pair[1])
        
    
    with torch.no_grad():
        model.eval()
        for v_pair in train_pairs[rubicon:]:
            vp,w2ivocab_valid,i2wvocab_valid = tensorFromPair(v_pair)
            valid_data = vp[0]
            valid_target = vp[1]
            valid_output = model(valid_data,valid_target)
            valid_loss = criterion(valid_output, valid_target)
            jaccard_score_valid += jaccard(outVector2wordList(valid_output[:-1],i2wvocab_valid),v_pair[1])
            loss_sum_valid +=valid_loss.item()
    
    
            
    loss2 = loss1
    loss1 = loss_sum_valid/len(train_pairs[rubicon:])
    
    print("Train loss: {:.4f},|Valid loss: {:.4f},|Train Jaccard: {:.4f},|Valid Jaccard: {:.4f},|Already gone: {}".
          format(loss_sum/len(train_pairs[:rubicon]),loss_sum_valid/len(train_pairs[rubicon:]),
        jaccard_score_train/len(train_pairs[:rubicon]),jaccard_score_valid/len(train_pairs[rubicon:]),
        timeSince(start)
    ))
    print('train_out:',outVector2wordList(output,i2wvocab))
    print('train_target:',t_pair[1])
    print('valid_out:',outVector2wordList(valid_output,i2wvocab_valid))
    print('valid_target:',v_pair[1])
    if loss2<loss1:
        indx+=1
        torch.save(model.state_dict(),'model_hs1024es300full'+str(epoch)+'epoch.pt')
        torch.save(encoder.state_dict(),'encoder_hs1024es300full'+str(epoch)+'epoch.pt')
        torch.save(decoder.state_dict(),'decoder_hs1024es300full'+str(epoch)+'epoch.pt')
    else:
        indx=0
    if indx==patience_s:
        break
    scheduler.step(loss_sum/len(train_pairs[:rubicon]))
    

In [ ]:
model.eval()
jacca = 0
for pair in train_pairs:    
    pt,w2i,i2w = tensorFromPair(pair)
    inp = pt[0]
    trg = pt[1]
    test_out = model(inp,trg)
    jacca += jaccard(pair[1],outVector2wordList(test_out[:-1],i2w))
print(jacca/len(train_pairs))

In [ ]:
def pred(encoder,decoder, source, teacher_force_ratio=0.5, max_length=35):
    sourse_len = source.shape[0]
    encoder_outputs =  torch.zeros(sourse_len, hidden_size*2, device=device).to(dtype=torch.float16,device=device)
    decoder_outputs = []
#     for ei in range(sourse_len):
    encoder_states, hidden = encoder(source)
#         encoder_outputs[ei] = encoder_states

#     x = source[0]
    x = sos_tensor
    for di in range(max_length):
        output, hidden = decoder(x, hidden, encoder_states)
        best_guess = output.argmax()
        
        if best_guess==eos_tensor:
#             decoder_outputs.append(vocab.index2word[best_guess.item()])
            break
        else:
            decoder_outputs.append(output)

    return torch.cat(decoder_outputs)

In [ ]:
encoder.eval()
decoder.eval()
l = []
for sentance in test_list:    
    w2i, idx_list, i2w = indexFromInput(sentance)
    inp = torch.tensor(idx_list).to(device)
    test_out = pred(encoder,decoder,inp)
    test_out = outVector2wordList(test_out[:-1],i2w)
    l.append(test_out)
# print(jacca/len(train_pairs))

In [ ]:
l = pd.Series(l)

In [ ]:
test['selected_text'] = l

In [ ]:
test[['textID','selected_text']].to_csv('submission.csv', index=False)
test

In [ ]:
# model.eval()
# note = 3
# pt,w2i,i2w = tensorFromPair(train_pairs[note])
# inp = pt[0]
# trg = pt[1]
# test_out = model(inp,trg)
# print(outVector2wordList(test_out,i2w))
# print(train_pairs[note])

In [ ]:
# model.eval()
# mnb=7
# zxc = model(training_pairs[mnb][0],training_pairs[mnb][1])
# print('input sentance:',train_pairs[mnb][0])
# print('out sentance:',outVector2wordList(zxc))
# print('target sentance:',targetVector2wordsList(training_pairs[mnb][1]))

In [ ]:
# def softmixer(vec):
#     return torch.exp(vec)/sum(torch.exp(vec))

In [ ]:
# def evaluation(encoder,decoder, source, teacher_force_ratio=0.5,target_vocab_size = vocab.n_words):
#     source = tensorFromIndex(source)
#     source_len = source.shape[0]
#     encoder_outputs =  torch.zeros(source_len, hidden_size*2, device=device).to(device)
#     attention_outputs = []
#     decoder_outputs = []
#     for ei in range(source_len):
#         encoder_states, hidden = encoder(source[ei])
#         encoder_outputs[ei] = encoder_states

# #     x = source[0]
#     x = sos_tensor
#     for di in range(source_len):
#         output, hidden, attention = decoder(x, hidden, encoder_outputs)
#         attention_outputs.append(attention.cpu().squeeze(1).detach().numpy())
#         best_guess = output.argmax()
#         x = best_guess
#         if best_guess.item()==eos_tensor.item():
# #             decoder_outputs.append(vocab.index2word[best_guess.item()])
#             break
#         else:
#             decoder_outputs.append(vocab.index2word[best_guess.item()])
        
#         max_tensor,_ = torch.tensor(attention_outputs).max(axis=0)
#         softmax_tensor = softmixer(max_tensor)
        
#     return decoder_outputs, softmax_tensor, torch.tensor(attention_outputs),hidden

In [ ]:
# change = 23
# print(test_list[change])
# out,smt, attn,h = evaluation(encoder,decoder,test_list[change])
# print(out)
# print(smt)
# print(attn)
# print(h)

In [ ]:
# change = 2
# print(pairs_list[change][0])
# out,smt, attn,h = evaluation(encoder,decoder,pairs_list[change][0])
# print(out)
# print(smt)
# print(attn)
# print(h)

In [ ]:
# change = 2
# print(pairs_list[change][0])
# out,smt, attn,h = evaluation(encoder,decoder,pairs_list[change][0])
# print(out)
# print(smt)
# print(attn)
# print(h)